### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

Всего должно получиться 18 комбинаций указанных параметров.

Создайте сравнительную таблицу по результатам проведенных тестов.

In [1]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
# Подключение модуля для работы с файлами
import os
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

E0000 00:00:1739791318.661202    6096 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739791318.715942    6096 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Распаковываем архив hw_light.zip в папку hw_light
# !unzip -q hw_light.zip

In [3]:
# Путь к директории с базой
base_dir = 'hw_light'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                    target_size=(img_height, img_width),
                                                    color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        elif patch == '3':
            y_train.append(1)
        else:
            y_train.append(2)

# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (302, 20, 20, 1)
Размер массива y_train (302,)


In [4]:
from itertools import product
import pandas as pd

CLASS_COUNT = 3
x_train = x_train.reshape(x_train.shape[0], -1)
y_train = utils.to_categorical(y_train, CLASS_COUNT)
neuron_count = [10, 100, 5000]
activation_functions = ["relu", "linear"]
batch_sizes = [10, 100, 1000]

In [5]:
results = []

for neurons, activation, batch_size in product(
    neuron_count, activation_functions, batch_sizes
):
    model = Sequential()
    model.add(Dense(neurons, input_dim=400, activation=activation))
    model.add(Dense(CLASS_COUNT, activation="softmax"))

    model.compile(
        loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"]
    )

    history = model.fit(
        x_train,
        y_train,
        epochs=15,
        batch_size=batch_size,
        verbose=0,
    )
    accuracy = history.history["accuracy"][-1]
    print(f"Количество нейронов: {neurons}, Функция активации: {activation}, Размер батча: {batch_size}, Точность: {accuracy}")

    results.append((neurons, activation, batch_size, accuracy))


df = pd.DataFrame(
    results,
    columns=["Количество нейтронов", "Функция активации", "Размер батча", "Точность"],
)

/home/aledonio/.cache/pypoetry/virtualenvs/global-cKEoLTwX-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1739791339.627047    6096 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1739791341.309141    9402 service.cc:148] XLA service 0x7fb6900043f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739791341.310621    9402 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
I0000 00:00:1739791341.467338    9402 cuda_dnn.cc:529] Lo

Количество нейронов: 10, Функция активации: relu, Размер батча: 10, Точность: 0.34437087178230286
Количество нейронов: 10, Функция активации: relu, Размер батча: 100, Точность: 0.42384105920791626
Количество нейронов: 10, Функция активации: relu, Размер батча: 1000, Точность: 0.4668874144554138
Количество нейронов: 10, Функция активации: linear, Размер батча: 10, Точность: 0.7450330853462219
Количество нейронов: 10, Функция активации: linear, Размер батча: 100, Точность: 0.5331125855445862
Количество нейронов: 10, Функция активации: linear, Размер батча: 1000, Точность: 0.4271523058414459
Количество нейронов: 100, Функция активации: relu, Размер батча: 10, Точность: 0.943708598613739
Количество нейронов: 100, Функция активации: relu, Размер батча: 100, Точность: 0.7218543291091919
Количество нейронов: 100, Функция активации: relu, Размер батча: 1000, Точность: 0.5960264801979065
Количество нейронов: 100, Функция активации: linear, Размер батча: 10, Точность: 0.7615894079208374
Количест

In [6]:
df

,Количество нейтронов,Функция активации,Размер батча,Точность
0,10,relu,10,0.344371
1,10,relu,100,0.423841
2,10,relu,1000,0.466887
3,10,linear,10,0.745033
4,10,linear,100,0.533113
5,10,linear,1000,0.427152
6,100,relu,10,0.943709
7,100,relu,100,0.721854
8,100,relu,1000,0.596026
9,100,linear,10,0.761589
